# __Applying PCM to ARGO data in Mediterranean Sea__

The aim of this notebook is to be an example of applying the PCM method to in-situ T-S profiles. The dataset is a selection of Argo profiles localted in Mediterranean sea. Notebook developped and run in __local__

Libraries needed:

In [ ]:
import xarray as xr
# xr.set_options(display_style='text')
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import argopy
# not at home
#argopy.set_options(cachedir='/home1/homedir5/perso/agarciaj/.cache/argopy',src='localftp',local_ftp='/home/ref-argo/gdac')
from argopy import DataFetcher as ArgoDataFetcher
argopy.set_options(cachedir='/home1/homedir5/perso/agarciaj/.cache/argopy',src='localftp',local_ftp='/home/ref-argo/gdac')
argo_loader = ArgoDataFetcher()
#argo_loader = ArgoDataFetcher(cache=True)
# trying parallel-requests
#argo_loader = ArgoDataFetcher(src='erddap', parallel=True, chunks={'wmo': 2})

import pyxpcm
from pyxpcm.models import pcm

from Plotter import Plotter #new class for plots in pyxpcm

from PIL import Image

## Getting data
***

We acces to data using argopy python library

<font color=red>TODO</font>: region option makes a square around Mediterranean sea, including some profiles in Black Sea and Atlantic Ocean --> Try to access data using WMO and profile number

In [ ]:
# only one year
ds = argo_loader.region([-6, 35, 30, 46, 0, 1000, '2012', '2013']).to_xarray()
ds

<font color=red>TODO</font>: try to use more than one year (memory limits ?)

In [ ]:
# trying yo use more than one year
#ds1 = argo_loader.region([30, 46, -6, 35, 0, 1000, '2011', '2012']).to_xarray()
#ds2 = argo_loader.region([30, 46, -6, 35, 0, 1000, '2012', '2013']).to_xarray()
#ds = xr.concat([ds1,ds2],dim='N_POINTS')
#ds

In [ ]:
dsp = ds.argo.point2profile()
dsp

In [ ]:
# interpolation to standard levels
z = np.arange(0.,1000,10.)
#z

with `z = np.arange(0.,200,1.)` interpolation is not working

In [ ]:
dsi = dsp.argo.interp_std_levels(z)
#dsi

Pressure should be negative to fit the model

In [ ]:
dsi['PRES_INTERPOLATED'] = -dsi.PRES_INTERPOLATED
dsi

In [ ]:
coords_list = list(dsi.coords.keys())
coords_list

In [ ]:
coords_list = list(dsi.coords.keys())

coords_dict = {}
for c in coords_list:
    axis_at = dsi[c].attrs.get('axis')
    print(axis_at)
    if axis_at == 'Y':
        #creates dictionary latitude: c
        coords_dict.update({'latitude': c})
    if axis_at == 'X':
        #creates dictionary longitude: c
        coords_dict.update({'longitude': c})
    if axis_at == 'T':
        #creates dictionay time: c
        coords_dict.update({'time': c})

#check number of variables in dict to assign data_type
if 'latitude' in coords_dict and 'longitude' in coords_dict:
    data_type = 'gridded'
else:
    data_type = 'profiles'
    
coords_dict


In [ ]:
ds

## Applying PCM
***

We creates a model with K classes and temperature or salinity or both as features

### Create PCM

In [ ]:
z = np.arange(0.,-1000,-10.)
# only temperature to try temporal representation
# pcm_features = {'temperature': z, 'salinity' : z}
pcm_features = {'temperature': z}
m = pcm(K=6, features=pcm_features)
m

### Fit model

In [ ]:
# only temperature to try temporal representation
# features_in_ds = {'temperature': 'TEMP', 'salinity': 'PSAL'}
features_in_ds = {'temperature': 'TEMP'}
features_zdim='PRES_INTERPOLATED'
m.fit(dsi, features=features_in_ds, dim=features_zdim)
m

<font color=red>__TODO__</font> : save all trained PCM in a folder

### Classify data

In [ ]:
m.predict(dsi, features=features_in_ds, dim=features_zdim, inplace=True);
#dsi

In [ ]:
m.predict_proba(dsi, features=features_in_ds, dim=features_zdim, inplace=True);
#dsi

In [ ]:
dsi = dsi.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of='TEMP', outname='TEMP' + '_Q', keep_attrs=True, inplace=True);
# dsi = dsi.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of='PSAL', outname='PSAL' + '_Q', keep_attrs=True, inplace=True)
#dsi

In [ ]:
#dsi

## Visualisation
***

Initialize Plotter

In [ ]:
P = Plotter(dsi, m, data_type = 'profiles')

#### 1. Vertical structure of classes

In [ ]:
P.vertical_structure(q_variable = 'TEMP_Q')
#plt.savefig('figures/vertical_struc_EX.png')

#### 2. Spatial distribution of classes

In [ ]:
P.spatial_distribution(proj=ccrs.PlateCarree(), extent=np.array([-10,40,30,56]) + np.array([-0.1,+0.1,-0.1,+0.1]), 
                       co = {'latitude':'LATITUDE', 'longitude':'LONGITUDE'})
#plt.savefig('figures/spatial_distr_EX.png')

#### 3. Posteriors

In [ ]:
P.plot_posteriors(proj=ccrs.PlateCarree(), extent=np.array([-10,40,30,56]) + np.array([-0.1,+0.1,-0.1,+0.1]), 
                       co = {'latitude':'LATITUDE', 'longitude':'LONGITUDE'})
#plt.savefig('figures/posteriors_EX.png')

#### 4. Temporal representation

Simple histogram

In [ ]:
fig, axes = plt.subplots()
dsi['PCM_LABELS'].plot.hist()
# y axis is number of profiles
axes.set_ylabel('Number of profiles')
#plt.savefig('figures/simple_hist_EX.png')
plt.show()

Bar plot by __month__

In [ ]:
P.temporal_distribution(time_variable = 'TIME', time_bins = 'month', pond = 'abs')
#plt.savefig('figures/temporal_distr_months_EX.png')

Bar plot by __month__ (__relative__ number of profiles per month)

In [ ]:
P.temporal_distribution(time_variable = 'TIME', time_bins = 'month', pond = 'rel')

Bar plot by __season__

In [ ]:
P.temporal_distribution(time_variable = 'TIME', time_bins = 'season', pond = 'rel')
plt.savefig('figures/temporal_distr_season_EX.png')